# Training and Detecting Object with GeoAI


## Install package
To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [ ]:
%pip install geoai-py segment-geospatial

## Import libraries

In [1]:
import geoai
import leafmap
from samgeo import tms_to_geotiff, split_raster

## Download satellite

Create an interactive map

In [ ]:
m = leafmap.Map(center=[24.699273257355753, 46.68045811333872], zoom=17, height="800px")
m.add_basemap("SATELLITE")
m

Create boundingbox and download image

In [ ]:
m.user_roi

In [ ]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [46.66867, 24.70191, 46.68044, 24.6916]

image = "data\Satellite\Test19.tif"
# tms_to_geotiff(output=image, bbox=bbox, zoom=21, source="Satellite", overwrite=True)   
leafmap.map_tiles_to_geotiff(
    output=image, bbox=bbox, zoom=19, source="Satellite", overwrite=True
) 

## Load training data

In [6]:
train_raster_path = r"data\POC3\Roi_Jun16.tif"
train_vector_path = r"data\POC3\Labels50.geojson"

## Visualize sample data

In [ ]:
geoai.view_vector_interactive(train_vector_path, tiles=train_raster_path)

## Create training data

In [ ]:
out_folder = r"results\Satellite\output"
tiles = geoai.export_geotiff_tiles(
    in_raster=train_raster_path,
    out_folder=out_folder,
    in_class_data=train_vector_path,
    tile_size=800,
    stride=400,
    buffer_radius=0,
)

## Train object detection model

In [ ]:
geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models20",
    num_channels=3,
    pretrained=True,
    batch_size=4,
    num_epochs=20,
    learning_rate=0.005,
    val_split=0.2,
)

## Run inference

In [16]:
out_folder = r"results\POC3\output"
test_raster_path = r"data\POC3\ROI_Jun16.tif"
masks_path = r"results\POC3\120Epochs_lr01\Label_area_120lr01_Jun16_prediction.tif"
model_path = f"{out_folder}/models120_lr01/best_model.pth"

In [ ]:
geoai.view_raster(test_raster_path)

In [17]:
geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=800,
    overlap=400,
    confidence_threshold=0.5,
    batch_size=4,
    num_channels=3,
)

Processing 100 windows with size 800x800 and overlap 400...


121it [04:45,  2.36s/it]                         


Inference completed in 285.31 seconds
Saved prediction to results\POC3\120Epochs_lr01\Label_area_120lr01_Jun16_prediction.tif


In [18]:
# masks_path = r"results\POC3\120Epochs\Test3_120_Jun16_prediction.tif"
output_path = r"results\POC3\120Epochs_lr01\Label_area_120lr01_Jun16_prediction.geojson"
gdf = geoai.orthogonalize(masks_path, output_path, epsilon=2)

Processing 670 features...


Converting features: 100%|██████████| 670/670 [00:10<00:00, 64.20shape/s]


Saving to results\POC3\120Epochs_lr01\Label_area_120lr01_Jun16_prediction.geojson...
Done!


## Visualize results

In [ ]:
geoai.view_vector_interactive(output_path, tiles=test_raster_path)